In [1]:
cd("/Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/spinfoam-critical-pts")

In [2]:
using LinearAlgebra, Combinatorics, Printf, Dates, PythonCall

sympy = pyimport("sympy")

include("GeometryTypes.jl")          # defines GeometryDataset, GeometryCollection
include("SimplexGeometry.jl")
include("SpinAlgebra.jl")
include("volume.jl")
include("TetraNormals.jl")
include("DihedralAngles.jl")
include("LorentzGroup.jl")
include("ThreeDTetra.jl")
include("Su2Su11FromBivector.jl")
include("XiFromSU.jl")
include("FaceNormals3D.jl")
include("KappaFromNormals.jl")
include("GeometryConsistency.jl")
include("GeometryPipeline.jl")
include("KappaOrientation.jl")
include("FourSimplexConnectivity.jl")
include("FaceXiMatching.jl")
include("FaceMatchingChecks.jl")
include("GaugeFixing.jl")
include("CriticalPoints.jl")
include("DefineSymbols.jl")
include("SolveVars.jl")
include("DefineAction.jl")

using .GeometryTypes: GeometryDataset, GeometryCollection
using .GeometryPipeline: run_geometry_pipeline
using .GeometryConsistency: check_sl2c_parallel_transport, check_so13_parallel_transport, check_closure_bivectors
using .KappaOrientation: fix_kappa_signs!
using .FourSimplexConnectivity: build_global_connectivity
using .FaceXiMatching: run_face_xi_matching
using .FaceMatchingChecks: check_all
using .GaugeFixingSU
using .CriticalPoints: compute_bdy_critical_data
using .DefineSymbols: run_define_variables
using .SolveVars: run_solver
using .DefineAction: vertexaction

In [3]:
bdypoints_all = [
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 0.0, 1.0, 1.0],
        [0.0, 1.0, 1.0, 1.0],
        [0.5, 1.0, 1.0, 1.0]
    ],
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 1.0, 1.0, 1.0],
        [0.5, 1.0, 1.0, 1.0],
        [0.0, 1.0, 0.0, 1.0]
    ]
]

2-element Vector{Vector{Vector{Float64}}}:
 [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 1.0], [0.0, 1.0, 1.0, 1.0], [0.5, 1.0, 1.0, 1.0]]
 [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.0, 1.0, 1.0, 1.0], [0.5, 1.0, 1.0, 1.0], [0.0, 1.0, 0.0, 1.0]]

In [4]:
geom = nothing
datasets = GeometryDataset[]
ns = length(bdypoints_all)

base_folder = "geom_output/new-data/"
mkpath(base_folder)

for s in 1:ns
    bdypoints = bdypoints_all[s]
    println("Boundary points for simplex $s:")

    ds = run_geometry_pipeline(bdypoints)  # GeometryDataset
    push!(datasets, ds)

end

Boundary points for simplex 1:
Boundary points for simplex 2:


In [5]:
geom = GeometryCollection(datasets);

In [6]:
four_simplices = [[1,2,3,4,5], [1,2,4,5,6]];

fix_kappa_signs!(four_simplices, geom);

build_global_connectivity(four_simplices, geom);

[1, 2]


In [7]:
conn = build_global_connectivity(four_simplices, geom)
push!(geom.connectivity, conn);

In [8]:
run_face_xi_matching(geom);

  SL(2,C) matrices updated:      ✓
  Boundary bivectors updated:    ✓
  boundary ξ variables updated:  ✓
  SU(2)/SU(1,1) elements updated: ✓
  SO(1,3) frames corrected:       ✓



In [9]:
check_all(geom)


   FACE–XI GEOMETRY CHECKS

✓ Face matching satisfied (max residual = 2.877074964446762e-16).
✓ SL(2,C) parallel transport satisfied (max residual = 5.650833432153205e-16).
✓ Closure condition satisfied (max residual = 2.5133742693021536e-16).

------------------------------
✓ All geometric consistency checks PASSED.
------------------------------



In [10]:
run_su2_su11_gauge_fix(geom);

In [11]:
run_define_variables(geom);

In [13]:
sol_vars, sol_bdry, γsym = run_solver(geom);

In [35]:
xi_mat1 = geom.varias[:xi_mat][1]
z_mat1 = geom.varias[:z_mat][1]
g_mat1 = geom.varias[:g_mat][1]
κdata = geom.simplex[1].kappa
sgndet = geom.simplex[1].sgndet
tetn0sign = geom.simplex[1].tetn0sign
tetareasign = geom.simplex[1].tetareasign
j_mat1 = geom.varias[:j_mat][1];

In [15]:
S = DefineAction.vertexaction(j_mat1, xi_mat1, z_mat1, g_mat1, κdata, sgndet, tetn0sign, tetareasign; γ = γsym);

In [19]:
S_vars = S.subs(sol_bdry).evalf();

In [22]:
S_simp = sympy.simplify(S_vars.subs(sol_vars).evalf());

In [23]:
dS = Dict{Py,Py}()

dS = Dict(v => sympy.diff(S_vars, v) for v in keys(sol_vars));

In [32]:
dS_eval = Dict{Py,Py}()

for (v, expr) in dS
    dS_eval[v] = sympy.simplify(expr.subs(sol_vars).subs(γsym, 1).evalf(chop=Py(true)))
end

In [35]:
dS_eval

Dict{Py, Py} with 44 entries:
  g_14_1 => 7.70976126622095e-16 - 1.18166432637496e-15*I
  g_13_3 => 1.00324740208192e-16 - 5.89128710554002e-16*I
  zc_115 => 0
  zc_145 => 0
  g_14_3 => -2.09464720553995e-15 + 3.18988837518071e-16*I
  g_13_1 => 1.84874738558396e-16 + 8.72226688043194e-16*I
  zc_135 => 8.07901292049095e-17*I
  z_125  => 0
  g_13_5 => -7.17068210953417e-17 + 1.03749570011676e-16*I
  g_12_3 => -1.36610221393735e-16 - 1.08001538955666e-16*I
  zc_123 => -6.0975101602719e-17 - 5.86955345907912e-16*I
  zc_124 => 9.57320221651102e-17
  g_14_6 => -1.78526836882852e-15 + 2.8035600061969e-15*I
  g_12_1 => -4.35908042523147e-16 - 1.14066786363235e-15*I
  zc_125 => 0
  g_13_4 => -7.77065027935197e-17 + 1.38415267295731e-15*I
  zc_113 => -8.35200584330549e-17 - 1.65329308898791e-16*I
  z_115  => 0
  g_12_6 => 5.14315821346033e-16 - 1.60532301247499e-16*I
  ⋮      => ⋮